In [125]:
# imoport libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.decomposition import NMF
import pickle
from fuzzywuzzy import process
import random

In [126]:
# import the data set of movie ratings 
df = pd.read_csv('../data/ml-latest-small/ratings.csv', na_values = 'Nan')
# import the data set of movie names
df_movie = pd.read_csv('../data/ml-latest-small/movies.csv', na_values = 'Nan')

In [127]:
# Split the movie name from movie year and apply it in the matrix
df_movie['year'] = df_movie.title.astype(str).str[-6:]
df_movie['title_new'] = df_movie.title.astype(str).str[:-6]

In [161]:
# make a frame with the Genres
# Try to zip columns with movie names
genres_movies = pd.DataFrame(list(zip(df_movie.genres, df_movie.title_new)),columns=['genres','movies'])
    # make column index
genres_movies.set_index('movies', inplace=True)
# Split multiple genres in several columns
genres_movies= genres_movies['genres'].str.split('|')
genres_movies.explode()


movies
Toy Story                        Adventure
Toy Story                        Animation
Toy Story                         Children
Toy Story                           Comedy
Toy Story                          Fantasy
                                   ...    
No Game No Life: Zero              Fantasy
Flint                                Drama
Bungo Stray Dogs: Dead Apple        Action
Bungo Stray Dogs: Dead Apple     Animation
Andrew Dice Clay: Dice Rules        Comedy
Name: genres, Length: 22084, dtype: object

In [129]:
# use pivot to make the matrix of movie rates
rates =df.pivot(index='userId',columns = 'movieId')
R = rates.rating

In [130]:
# Try to zip columns with movie names
#new_columns = dict(zip(df.movieId,df_movie.title))
R.rename(columns=dict(zip(df_movie["movieId"], df_movie["title_new"])),inplace = True)
R.head()

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


movieId,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Tom and Huck,Sudden Death,GoldenEye,...,Gintama: The Movie,anohana: The Flower We Saw That Day - The Movie,Silver Spoon,Love Live! The School Idol Movie,Jon Stewart Has Left the Building,Black Butler: Book of the Atlantic,No Game No Life: Zero,Flint,Bungo Stray Dogs: Dead Apple,Andrew Dice Clay: Dice Rules
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [131]:
# fill in the NaN values with zeros
# impute the nan with 0:
R = R.fillna(value=0)
R.sample(5)

movieId,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Tom and Huck,Sudden Death,GoldenEye,...,Gintama: The Movie,anohana: The Flower We Saw That Day - The Movie,Silver Spoon,Love Live! The School Idol Movie,Jon Stewart Has Left the Building,Black Butler: Book of the Atlantic,No Game No Life: Zero,Flint,Bungo Stray Dogs: Dead Apple,Andrew Dice Clay: Dice Rules
userId,,,,,,,,,,,,,,,,,,,,,
441,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
182,4.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,3.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
355,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [132]:
# Create the functions for the cosine similarity table
def cos_sim(vec1, vec2):
    """function to calcualte the cosine similarity between two vectors""" 
    num = np.dot(vec1, vec2)
    denom = np.sqrt(np.dot(vec1, vec1)) * np.sqrt(np.dot(vec2, vec2))
    return num / denom

def cos_sim_table(R):
    data = []
    for user1 in R.index:
        row = []
        for user2 in R.index:
            cs = cos_sim(R.loc[user1],R.loc[user2])
            row.append(cs)
        data.append(row)
    table = pd.DataFrame(data,index=R.index, columns=R.index)
    return table

In [133]:
from sklearn.metrics.pairwise import cosine_similarity

In [134]:
# Returns numpy array:
cosine_similarity(R)

array([[1.        , 0.02728287, 0.05972026, ..., 0.29109737, 0.09357193,
        0.14532081],
       [0.02728287, 1.        , 0.        , ..., 0.04621095, 0.0275654 ,
        0.10242675],
       [0.05972026, 0.        , 1.        , ..., 0.02112846, 0.        ,
        0.03211875],
       ...,
       [0.29109737, 0.04621095, 0.02112846, ..., 1.        , 0.12199271,
        0.32205486],
       [0.09357193, 0.0275654 , 0.        , ..., 0.12199271, 1.        ,
        0.05322546],
       [0.14532081, 0.10242675, 0.03211875, ..., 0.32205486, 0.05322546,
        1.        ]])

In [135]:
# We can turn this into a dataframe:
cos_sim_table = pd.DataFrame(cosine_similarity(R),index=R.index, columns = R.index)
cos_sim_table

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.027283,0.059720,0.194395,0.129080,0.128152,0.158744,0.136968,0.064263,0.016875,...,0.080554,0.164455,0.221486,0.070669,0.153625,0.164191,0.269389,0.291097,0.093572,0.145321
2,0.027283,1.000000,0.000000,0.003726,0.016614,0.025333,0.027585,0.027257,0.000000,0.067445,...,0.202671,0.016866,0.011997,0.000000,0.000000,0.028429,0.012948,0.046211,0.027565,0.102427
3,0.059720,0.000000,1.000000,0.002251,0.005020,0.003936,0.000000,0.004941,0.000000,0.000000,...,0.005048,0.004892,0.024992,0.000000,0.010694,0.012993,0.019247,0.021128,0.000000,0.032119
4,0.194395,0.003726,0.002251,1.000000,0.128659,0.088491,0.115120,0.062969,0.011361,0.031163,...,0.085938,0.128273,0.307973,0.052985,0.084584,0.200395,0.131746,0.149858,0.032198,0.107683
5,0.129080,0.016614,0.005020,0.128659,1.000000,0.300349,0.108342,0.429075,0.000000,0.030611,...,0.068048,0.418747,0.110148,0.258773,0.148758,0.106435,0.152866,0.135535,0.261232,0.060792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.164191,0.028429,0.012993,0.200395,0.106435,0.102123,0.200035,0.099388,0.075898,0.088963,...,0.178084,0.116534,0.300669,0.066032,0.148141,1.000000,0.153063,0.262558,0.069622,0.201104
607,0.269389,0.012948,0.019247,0.131746,0.152866,0.162182,0.186114,0.185142,0.011844,0.010451,...,0.092525,0.199910,0.203540,0.137834,0.118780,0.153063,1.000000,0.283081,0.149190,0.139114
608,0.291097,0.046211,0.021128,0.149858,0.135535,0.178809,0.323541,0.187233,0.100435,0.077424,...,0.158355,0.197514,0.232771,0.155306,0.178142,0.262558,0.283081,1.000000,0.121993,0.322055


In [158]:
def create_new_user(R , k = 5):
    """
    In this function we initialize a new user according to a list of movies given 
    in the R Data-Frame. The unseen movies are considered as zero rating.
    This function is used for the cosine similarity suggesting function.
    k is the number of movies to ask for a rating
    
    """
    # initiate a new user
    new_user = np.zeros_like(R.columns)
    films = R.columns
    rand_movies = random.choices(films,k=k)
    user_votes_ran_movies = []
    for idx, movies in enumerate(rand_movies):
        user_votes_ran_movies.append(input(f'Give me the rating for the movie {movies}.'))
    # make a list out of the 
    new_user_ratings = pd.DataFrame(list(zip(rand_movies, user_votes_ran_movies)),columns=['movie','rating'])
    # make column index
    new_user_ratings.set_index('movie', inplace=True)
    for idx,item in enumerate(films):
        if new_user_ratings.get(item):   # returns the value for key if the key is in dictionary
            new_user[idx] = new_user_ratings[item]
    # new user dataframe
    new_user_rates = pd.DataFrame([new_user], index=['new_user'],columns = films)
    R_new = new_user_rates.append(R)
    return R_new

In [159]:
def cos_sim_tab(R):
    cosine_similarity(R)
    # We can turn this into a dataframe:
    cos_sim_table = pd.DataFrame(cosine_similarity(R),index=R.index, columns = R.index)
    return cos_sim_table

In [ ]:
def movies_user_input():
    """
    ask user about the movies he likes
    """
    selections = []
    selection = []
    user_movies = []
    while selection != 'end':
        selection = input('Give me your top 5 movies.')
        selections.append(selection)
    for i in selections:
        user_movies.append(process.extractOne(str(i), df_movie.title_new))
    return selections[:-1]

In [160]:
new_user_rates = create_new_user(R,5)
cos_sim_table = cos_sim_tab(new_user_rates)

In [ ]:
def recomm(R,cos_sim_table,):
    R_t = R.T
    unseen_movies = list(new_user_rates == 0)
    cos_sim_table[active_user].sort_values(ascending=False)




In [150]:
# create a list of unseen movies for this user
unseen_movies = list(new_user_rates == 0)
unseen_movies

['Toy Story ',
 'Jumanji ',
 'Grumpier Old Men ',
 'Waiting to Exhale ',
 'Father of the Bride Part II ',
 'Heat ',
 'Sabrina ',
 'Tom and Huck ',
 'Sudden Death ',
 'GoldenEye ',
 'American President, The ',
 'Dracula: Dead and Loving It ',
 'Balto ',
 'Nixon ',
 'Cutthroat Island ',
 'Casino ',
 'Sense and Sensibility ',
 'Four Rooms ',
 'Ace Ventura: When Nature Calls ',
 'Money Train ',
 'Get Shorty ',
 'Copycat ',
 'Assassins ',
 'Powder ',
 'Leaving Las Vegas ',
 'Othello ',
 'Now and Then ',
 'Persuasion ',
 'City of Lost Children, The (Cité des enfants perdus, La) ',
 'Shanghai Triad (Yao a yao yao dao waipo qiao) ',
 'Dangerous Minds ',
 'Twelve Monkeys (a.k.a. 12 Monkeys) ',
 'Babe ',
 'Dead Man Walking ',
 'It Takes Two ',
 'Clueless ',
 'Cry, the Beloved Country ',
 'Richard III ',
 'Dead Presidents ',
 'Restoration ',
 'Mortal Kombat ',
 'To Die For ',
 'How to Make an American Quilt ',
 'Seven (a.k.a. Se7en) ',
 'Pocahontas ',
 'When Night Is Falling ',
 'Usual Suspects, 

In [ ]:
cos_sim_table[active_user].sort_values(ascending=False)